In [34]:
import os
import sys
import json
import time
import datetime
import pandas as pd

from dotenv import load_dotenv

from intervals import Intervals

load_dotenv()


ATHLETE_ID = ""
API_KEY = ""

try:
    ATHLETE_ID = os.environ['ATHLETE_ID']
    API_KEY = os.environ['API_KEY']
except KeyError:
    print("Please set ATHLETE_ID and API_KEY in .env file")
    sys.exit(1)


print(ATHLETE_ID)
print(API_KEY)

i171109
7hqhcubmp6ikkulwi4zanoj88


In [40]:
icu = Intervals(ATHLETE_ID, API_KEY)

START_DATE = "01/01/2024"
WELLNESS_COLS = [
    "ctl_start",
    "atl_start",
    "date",
    "weight",
    "watt_kg",
]

start_date = datetime.datetime.strptime(START_DATE, "%d/%m/%Y").date()
today_date = datetime.date.today()


def fetch_wellness(start_date, end_date):
    data = icu.wellness(start_date, today_date)
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["id"])
    df = df.drop(columns=["id"])
    df = df.sort_values(by="date")
    df['eftp'] = df['sportInfo'].apply(lambda x: x[0]['eftp'] if len(x) > 0 else None)
    df = df.drop(columns=["sportInfo"])

    df['weight'] = df['weight'].ffill().bfill()
    df['eftp'] = df['eftp'].ffill().bfill()

    df['ctl_start'] = df['ctl'].shift(1)
    df['atl_start'] = df['atl'].shift(1)

    df['watt_kg'] = df['eftp'] / df['weight']

    return df[WELLNESS_COLS]

tic = time.time()
df = fetch_wellness(start_date, today_date)
toc = time.time()
print(f"Time taken: {toc-tic:.2f}s")

Time taken: 0.36s
